<a href="https://colab.research.google.com/github/acdc2019/algo-trading/blob/main/python/notebooks/strategies/rsi_strategy/RSIStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **RSI Strategy**
## **15 mins strategy**
* Step 1: A 15 min candle has to close above 70 RSI value.
* Step 2: Above High of this candle Buy signal is generated.
* Step 3: Stop Loss of the trade has to be below the earlier 15 min candles low.
* Step 4: Volume of the candle in which buy signal is generated has to be atleast 2 to 3 times of earlier 5 candles (***Q: average of earlier 5 candle volume?***)
* Step 5: Hourly candles RSI should be above 50 for confirmation.
* Step 6: In the last 5 candles the stock movement should not be more than 4% of the day movement
* Step 7: If Stop Loss of the candle is less than 6000 INR then the trade should be executed

#### **Strategy Parameters**
* window_start, window_end: Dates between which to look for signal
* rsi_15min = 70
* rsi_60min = 50
* stop_loss = 6000
* back_candles = 5
* volume_multiple = 2 or 3 times
* daily_movement_pct = 4

Install libs

In [1]:
!pip install ta
!pip install pandas==1.3.5
!pip install plotly
!pip install numpy

In [2]:
!wget https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77A6U5DAJWZBHECHBRSLBYMGGM -O helpers.py

--2021-12-22 11:29:58--  https://raw.githubusercontent.com/acdc2019/algo-trading/main/python/strategy/helpers.py?token=ANF77A6U5DAJWZBHECHBRSLBYMGGM
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1177 (1.1K) [text/plain]
Saving to: ‘helpers.py’

helpers.py          100%[===================>]   1.15K  --.-KB/s    in 0s      

2021-12-22 11:29:58 (52.6 MB/s) - ‘helpers.py’ saved [1177/1177]



Import Libs

In [6]:
import helpers
import pandas as pd
from ta.momentum import RSIIndicator
from plotly.subplots import make_subplots
import numpy as np

In [7]:
def get_tanaji_pct(df, index, n, high):
  prev_candles = helpers.get_previous_candles(df, index, n)
  min_low = prev_candles['Low'].min()
  daily_movement = high - min_low
  tanaji_pct = (daily_movement/min_low)*100
  return tanaji_pct

def get_hourly_rsi(df_15min, index):
  df_15min_temp = df_15min[:index]
  #print(index)
  #print(df_15min_temp.tail(10))
  df_60min_o = df_15min_temp['Open'].resample('60Min', offset='30Min').apply({'Open': 'first'})
  df_60min_h = df_15min_temp['High'].resample('60Min', offset='30Min').apply({'High': 'max'})
  df_60min_l = df_15min_temp['Low'].resample('60Min', offset='30Min').apply({'Low': 'min'})
  df_60min_c = df_15min_temp['Close'].resample('60Min', offset='30Min').apply({'Close': 'last'})
  df_60min_vol = df_15min_temp['Volume'].resample('60Min', offset='30Min').apply({'Volume': 'sum'})
  
  df_60min = pd.concat([df_60min_o, df_60min_h, df_60min_l, df_60min_c, df_60min_vol], axis=1)
  df_60min.dropna(subset=['Open'], inplace=True)

  rsi = RSIIndicator(df_60min['Close']).rsi()
  df_60min = df_60min.assign(rsi=rsi.values)
  return df_60min

## Load 15 min and 60 min stock data and calculate RSI

In [27]:
file_15min = '/content/stock_data/LT-HIST.csv'

df_15min = pd.read_csv(file_15min, parse_dates=['Date'], index_col=['Date'])
rsi = RSIIndicator(df_15min['Close']).rsi()
df_15min = df_15min.assign(rsi=rsi.values)
df_15min.tail()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-12-17 14:15:00+05:30,1870.00,1870.80,1857.00,1857.40,68357,53.205167
2021-12-17 14:30:00+05:30,1857.45,1859.45,1846.15,1847.50,87319,46.057840
2021-12-17 14:45:00+05:30,1847.95,1848.35,1842.00,1847.25,159813,45.890192
2021-12-17 15:00:00+05:30,1847.20,1851.15,1841.70,1846.80,382071,45.568664
2021-12-17 15:15:00+05:30,1847.35,1852.00,1843.10,1846.00,355192,44.965492


## Set Strategy Parameters

In [28]:
window_start = '2021-01-01 00:00:00'
window_end = '2021-12-15 00:00:00'
rsi_15min = 70
rsi_60min = 50
lot_size = 575
stop_loss = 6000
back_candles = 5
volume_multiple = 2 # 2 or 3 times
daily_movement_pct = 4


## Actual Strategy Implementation

### Step 1: A 15 min candle has to close above 70 RSI value
Get all the candles in the window that close above required RSI value

In [29]:
curr_window_df = df_15min[window_start:window_end]

rsi_filter = (curr_window_df['rsi'] > rsi_15min)
df = curr_window_df[rsi_filter]

bullish_filter = (df['Close'] > df['Open'])
df = df[bullish_filter]

df.head()


,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-04 09:30:00+05:30,1306.90,1314.80,1305.3,1311.00,282837,73.685682
2021-01-04 09:45:00+05:30,1310.70,1318.90,1308.4,1317.00,339204,78.257209
2021-01-04 10:30:00+05:30,1312.20,1316.85,1311.5,1315.65,114504,70.821836
2021-01-04 10:45:00+05:30,1315.50,1322.50,1315.0,1320.10,421529,74.298579
2021-01-07 09:45:00+05:30,1339.85,1342.00,1338.0,1341.80,170195,70.247167



### Step 4: Volume check

In [30]:
temp_df = pd.DataFrame(columns=['Open','High','Low','Close','Volume','rsi','MeanVolume'])
temp_df.index.name = 'Date'
back_candles_df = pd.DataFrame()
for index, row in df.iterrows():
  # Get mean volume of previous candles
  back_candles_df = helpers.get_previous_candles(curr_window_df, index, back_candles)
  mean_volume = back_candles_df['Volume'].mean()
  row['MeanVolume'] = mean_volume

  # Compare volume against previous mean volume
  if(row['Volume'] > volume_multiple*mean_volume):
    # print('Current Volume:', row['Volume'], 'greather than 2 times mean volume', mean_volume)
    temp_df.loc[index] = row

df = temp_df[['Open','High','Low','Close','Volume','rsi']]
temp_df.head()

,Open,High,Low,Close,Volume,rsi,MeanVolume
Date,,,,,,,
2021-01-04 09:30:00+05:30,1306.9,1314.8,1305.30,1311.0,282837.0,73.685682,114209.8
2021-01-04 09:45:00+05:30,1310.7,1318.9,1308.40,1317.0,339204.0,78.257209,162005.2
2021-01-04 10:45:00+05:30,1315.5,1322.5,1315.00,1320.1,421529.0,74.298579,182528.8
2021-01-08 09:15:00+05:30,1349.8,1359.4,1347.00,1356.0,448562.0,71.103308,157875.0
2021-01-13 09:15:00+05:30,1354.0,1368.3,1350.65,1367.0,432065.0,74.699941,148660.4


### Step 5: Hourly candle RSI check

In [31]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  df_60min = get_hourly_rsi(df_15min, index)
  #print(df_60min.tail(15))
  
  rsi = df_60min.iloc[-1]['rsi']
  if(rsi > rsi_60min):
    # This 15min candle is eligible for signal
    print('Hourly candle RSI is greater than 50..', rsi)
    temp_df.loc[index] = row

df = temp_df
df.head()


Hourly candle RSI is greater than 50.. 65.31395843110492
Hourly candle RSI is greater than 50.. 68.11586670184423
Hourly candle RSI is greater than 50.. 69.63176863902873
Hourly candle RSI is greater than 50.. 73.63862066437838
Hourly candle RSI is greater than 50.. 63.466013239832066
Hourly candle RSI is greater than 50.. 67.62929825632719
Hourly candle RSI is greater than 50.. 72.56330671559138
Hourly candle RSI is greater than 50.. 84.3246268620165
Hourly candle RSI is greater than 50.. 76.65871201412651
Hourly candle RSI is greater than 50.. 64.78172162559386
Hourly candle RSI is greater than 50.. 60.99323197415336
Hourly candle RSI is greater than 50.. 70.75463088457606
Hourly candle RSI is greater than 50.. 63.054559232785756
Hourly candle RSI is greater than 50.. 61.37894509551944
Hourly candle RSI is greater than 50.. 72.1106913377488
Hourly candle RSI is greater than 50.. 67.70149765706955
Hourly candle RSI is greater than 50.. 67.97496206245087
Hourly candle RSI is greater th

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-04 09:30:00+05:30,1306.9,1314.8,1305.30,1311.0,282837.0,73.685682
2021-01-04 09:45:00+05:30,1310.7,1318.9,1308.40,1317.0,339204.0,78.257209
2021-01-04 10:45:00+05:30,1315.5,1322.5,1315.00,1320.1,421529.0,74.298579
2021-01-08 09:15:00+05:30,1349.8,1359.4,1347.00,1356.0,448562.0,71.103308
2021-01-13 09:15:00+05:30,1354.0,1368.3,1350.65,1367.0,432065.0,74.699941


### Step 6: Stock movement check

In [32]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  tanaji_pct = get_tanaji_pct(df_15min, index, 5, row['High'])
  print('tanaji_pct:', tanaji_pct)
  if(tanaji_pct < daily_movement_pct):
    temp_df.loc[index] = row

df = temp_df
temp_df

tanaji_pct: 1.5171987800640747
tanaji_pct: 1.8337644288306372
tanaji_pct: 1.3177047422048607
tanaji_pct: 1.6753926701570747
tanaji_pct: 1.5662114014251711
tanaji_pct: 2.8582945509179525
tanaji_pct: 4.287441826376928
tanaji_pct: 6.980711649606573
tanaji_pct: 1.4658352649276907
tanaji_pct: 2.040816326530609
tanaji_pct: 2.369411206399786
tanaji_pct: 2.81393819855358
tanaji_pct: 1.1041732811232294
tanaji_pct: 1.4889547312755433
tanaji_pct: 1.4329167735551862
tanaji_pct: 1.5554495762867342
tanaji_pct: 2.30531856961336
tanaji_pct: 2.7264476311490258
tanaji_pct: 2.82988181719613
tanaji_pct: 0.849249540711983
tanaji_pct: 1.2145201123177403
tanaji_pct: 1.609969686763226
tanaji_pct: 2.054563826204109
tanaji_pct: 3.359112306657703
tanaji_pct: 0.9501965923984272
tanaji_pct: 0.8740921653062478
tanaji_pct: 1.6340184631219958
tanaji_pct: 1.5920295809367295
tanaji_pct: 1.9218025182239893
tanaji_pct: 1.146488469601677
tanaji_pct: 4.019350021103217
tanaji_pct: 5.191389889938641
tanaji_pct: 1.95273631840

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-01-04 09:30:00+05:30,1306.90,1314.8,1305.30,1311.00,282837.0,73.685682
2021-01-04 09:45:00+05:30,1310.70,1318.9,1308.40,1317.00,339204.0,78.257209
2021-01-04 10:45:00+05:30,1315.50,1322.5,1315.00,1320.10,421529.0,74.298579
2021-01-08 09:15:00+05:30,1349.80,1359.4,1347.00,1356.00,448562.0,71.103308
2021-01-13 09:15:00+05:30,1354.00,1368.3,1350.65,1367.00,432065.0,74.699941
...,...,...,...,...,...,...
2021-11-03 14:30:00+05:30,1895.00,1902.0,1892.75,1898.20,1391573.0,71.440014
2021-11-11 09:45:00+05:30,1959.35,1979.4,1959.10,1975.85,877677.0,75.527908
2021-12-03 09:15:00+05:30,1799.95,1837.1,1799.05,1835.00,500391.0,83.728661


### Step 7: Stop Loss check

In [33]:
temp_df = pd.DataFrame(columns=df.columns)
temp_df.index.name = 'Date'

for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  prev_low = prev_candle.iloc[0]['Low']
  if((row['High'] - prev_low)*lot_size > stop_loss):
    print(index, 'Stop Loss greater than 6000 INR. Do not trade', (row['High'] - prev_low)*lot_size)
  else:
    print(index, 'Stop Loss within range', (row['High'] - prev_low)*lot_size)
    temp_df.loc[index] = row

df = temp_df
df

2021-01-04 09:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 7359.999999999974
2021-01-04 09:45:00+05:30 Stop Loss greater than 6000 INR. Do not trade 7820.000000000078
2021-01-04 10:45:00+05:30 Stop Loss greater than 6000 INR. Do not trade 6325.0
2021-01-08 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 12851.250000000078
2021-01-13 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 12132.499999999947
2021-02-01 13:00:00+05:30 Stop Loss greater than 6000 INR. Do not trade 22080.00000000005
2021-02-03 11:45:00+05:30 Stop Loss greater than 6000 INR. Do not trade 12908.750000000025
2021-02-16 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 17997.499999999975
2021-03-03 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 14001.249999999947
2021-03-12 09:15:00+05:30 Stop Loss greater than 6000 INR. Do not trade 24609.999999999975
2021-04-08 12:30:00+05:30 Stop Loss greater than 6000 INR. Do not trade 8682.499999999947
2021-04-27 09:3

,Open,High,Low,Close,Volume,rsi
Date,,,,,,
2021-05-10 12:30:00+05:30,1376.05,1384.15,1375.35,1382.20,379209.0,75.432806
2021-05-24 12:45:00+05:30,1449.00,1454.70,1449.00,1453.50,319229.0,74.804227
2021-06-04 12:30:00+05:30,1535.40,1540.50,1532.55,1539.00,463008.0,76.427624
2021-08-12 15:00:00+05:30,1618.35,1623.95,1618.15,1623.20,196334.0,74.449820
2021-08-12 15:15:00+05:30,1623.35,1627.00,1621.00,1624.00,277427.0,75.075732
2021-09-14 12:30:00+05:30,1686.90,1692.00,1685.00,1691.70,58014.0,72.685692
2021-09-14 15:00:00+05:30,1693.35,1698.50,1692.45,1697.65,123115.0,74.531109
2021-09-15 13:45:00+05:30,1717.40,1724.60,1716.35,1721.75,163899.0,74.746458
2021-11-03 14:30:00+05:30,1895.00,1902.00,1892.75,1898.20,1391573.0,71.440014


## **Final Signal**

In [34]:
for index, row in df.iterrows():
  prev_candle = helpers.get_previous_candles(df_15min, index, 1)
  stop_loss = prev_candle.iloc[0]['Low']
  print('For candle:', index, ', rsi:', row['rsi'], 'Entry Price', row['High'], 'Stop Loss:', stop_loss)

For candle: 2021-05-10 12:30:00+05:30 , rsi: 75.43280612962587 Entry Price 1384.15 Stop Loss: 1373.9
For candle: 2021-05-24 12:45:00+05:30 , rsi: 74.80422662479948 Entry Price 1454.7 Stop Loss: 1445.5
For candle: 2021-06-04 12:30:00+05:30 , rsi: 76.42762372044584 Entry Price 1540.5 Stop Loss: 1532.0
For candle: 2021-08-12 15:00:00+05:30 , rsi: 74.4498196092276 Entry Price 1623.95 Stop Loss: 1615.25
For candle: 2021-08-12 15:15:00+05:30 , rsi: 75.07573248286619 Entry Price 1627.0 Stop Loss: 1618.15
For candle: 2021-09-14 12:30:00+05:30 , rsi: 72.68569190751958 Entry Price 1692.0 Stop Loss: 1685.0
For candle: 2021-09-14 15:00:00+05:30 , rsi: 74.53110864841926 Entry Price 1698.5 Stop Loss: 1690.0
For candle: 2021-09-15 13:45:00+05:30 , rsi: 74.74645808670509 Entry Price 1724.6 Stop Loss: 1715.5
For candle: 2021-11-03 14:30:00+05:30 , rsi: 71.44001412553925 Entry Price 1902.0 Stop Loss: 1894.0
For candle: 2021-12-09 12:30:00+05:30 , rsi: 77.46325689515533 Entry Price 1865.3 Stop Loss: 1855

In [35]:
import plotly.graph_objects as go
import plotly.offline as py

back=5
next=50
sig_param_col1 = ['Time','Entry INR','StopLoss INR','Daily Mov %','Volume','Mean Vol','15Min RSI','Horuly RSI']
for index, row in df.iterrows():
  prev_candles = helpers.get_previous_candles(df_15min, index, back, True)
  next_candles = helpers.get_next_candles(df_15min, index, next)
  
  candles = pd.concat([prev_candles, next_candles])
  candles['DateStr'] = candles.index.strftime('%d-%m %H:%M')

  # Get all strategy params for this signal
  stop_loss_candle = helpers.get_previous_candles(df_15min, index, 1)
  vol_back_candles_df = helpers.get_previous_candles(df_15min
                                                 , index, back_candles)
  sig_stop_loss = stop_loss_candle.iloc[0]['Low']
  sig_entry = candles['High'][back]
  sig_daily_mov_pct = np.round(get_tanaji_pct(df_15min, index, 5, row['High']),2)
  sig_rsi = np.round(candles['rsi'][back],2)
  sig_hourly_rsi = np.round(get_hourly_rsi(df_15min, index).iloc[-1]['rsi'],2)
  sig_mean_volume = vol_back_candles_df['Volume'].mean()
  sig_volume = row['Volume']

  fig = make_subplots(rows=2, cols=2, shared_xaxes=False,
               subplot_titles=('OHLC', 'Signal Params', 'Volume & RSI',''), 
               vertical_spacing=0.1, 
               horizontal_spacing=0.01,
               row_width=[0.5, 0.7],
               column_widths=[0.8,0.2],
               specs=[[{"secondary_y": False, "type": "candlestick"},{"secondary_y": False, "type":"table"}], 
                      [{"secondary_y": True}, {"secondary_y": False}]])
    
  fig.add_trace(go.Candlestick(x=candles['DateStr'],
                     open=candles['Open'],
                     high=candles['High'],
                     low=candles['Low'],
                     close=candles['Close'],
                     name='Signal Chart',
                     increasing_line_color='yellow',
                     increasing_fillcolor='yellow',
                     decreasing_line_color='red',
                     decreasing_fillcolor='red',),
                     row=1,col=1)
  
  fig.add_annotation(x=back,y=sig_entry,
                     text='Signal')
  
  # Position Entry Point
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_entry, y1=sig_entry, 
                line=dict(color='Green'),
                row=1,col=1)
  
  # Position Stop Loss
  fig.add_shape(type='line', 
                x0=-1,x1=back+next+2,
                y0=sig_stop_loss, y1=sig_stop_loss, 
                line=dict(color='Red'),
                row=1,col=1)

  # Signal Parameters Table
  fig.add_trace(go.Table(header=dict(values=['Param','Value'],
                                     line_color='white',
                                     fill_color='darkslategray',
                                     align='left'),
                         cells=dict(values=[sig_param_col1,
                                           [index.strftime('%d-%m %H:%M'), sig_entry, 
                                            sig_stop_loss, sig_daily_mov_pct, sig_volume, 
                                            sig_mean_volume, sig_rsi, sig_hourly_rsi]],
                                    line_color='white',
                                    fill_color='black',
                                    align='left')),
                row=1, col=2)

  # RSI
  fig.add_trace(go.Scatter(x=candles['DateStr'], y=candles['rsi'], name='rsi',            
           marker_color='Cyan'),
           row=2, col=1)
  
  # Volume
  fig.add_trace(go.Bar(x=candles['DateStr'], y=candles['Volume'], name='Volume',            
           marker_color='rgb(55, 55, 109)',
           width=np.array([0.5]*candles.size)),
           secondary_y=True,
          row=2, col=1)
  
  fig.add_annotation(x=back,y=sig_rsi,
                     text='Signal',row=2, col=1)
  
  fig.add_shape(type='line', x0=-1,x1=back+next+2,y0=rsi_15min, y1=rsi_15min, 
                line=dict(color='Green'),
                row=2,col=1)

  fig.update_xaxes(type='category', rangeslider=dict(visible=False))
  fig.update_xaxes(showgrid=False, nticks=5)
  fig.update_yaxes(showgrid=False)
  fig.update_layout(
    title='Signal generated for RSI 15mins Strategy',
    title_x = 0.5,
    autosize=False,
    width=1450,
    height=750,
    plot_bgcolor='rgb(5,5,5)',
    paper_bgcolor='rgb(0,0,0)',
    font_color='white')

  py.iplot(fig)
  